In [1]:
import os
from pprint import pprint

from dotenv import load_dotenv
from pyzotero import zotero

In [2]:
load_dotenv()
zot = zotero.Zotero(os.getenv("ZOTERO_ID"), "user", os.getenv("ZOTERO_API_KEY"), local=False)

In [3]:
zot.count_items()

4226

In [4]:
library = zot.collections()
collection_dict = {}
for collection in library:
    collection_dict[collection['data']['name']] = collection['data']['key']
pprint(collection_dict)

{'Can neural mass models accurately capture synaptic plasticity mechanisms - 13 Jul 2025': 'WKYIZNNF',
 'Cited in - Neuroplasticity Thread': '4ZH73WBS',
 'Cited in entropyThread - 01 Aug 2025': 'WPXG7R5M',
 'ComplexityEntropyChaos': 'ZVCGI7JK',
 'ConsensusCitation - 31 Jul 2025': 'KMSLSSL7',
 'EEG': 'IK26UWYI',
 'EEG_For_Meditation': '752GWEEQ',
 'FundamentalBooks': 'I6MYWKWQ',
 'How are neuroplasticity, learning and recovery modeleld in state of the art brain models like Neural Mass Models - 13 Jul 2025': '88AFQG3V',
 'How can mechanistic synaptic plasticity rules be computationally scaled and integrated into neural mass models to create biologically realistic, adaptive brain network simulations that capture learning, recovery, and pathological dynamics - 13 Jul 25': 'CL6BNXEM',
 'IIT': 'PBCQNZV2',
 'MasterThesis': 'VEG5XGN8',
 'Mystical Entropy': 'PH7L637T',
 'Neuroplasticity+NMM-review': 'QPEWATYL',
 'NotionSync': 'B6GWT9RN',
 'PCI-From-Resting-State-Reconstruction': '574RX5AA',
 'P

In [5]:
COLLECTION_ID = collection_dict.get('EEG')
item_dict = {}
only_parent_items = True
for item in zot.collection_items(COLLECTION_ID):
    if not (item_data := item.get('data', None)):
        continue
    if only_parent_items and len(item_data.get('parentItem', '')) == 0:
        item_id = item.get('key')
        item_name = item.get('data', {}).get('title')
        item_dict[item_name] = item_id
pprint(item_dict)

{'A Tutorial on Independent Component Analysis': 'IJPXFJC5',
 'A Tutorial on Principal Component Analysis': 'KPR5XSGI',
 'DISCOVER-EEG: an open, fully automated EEG pipeline for biomarker discovery in clinical neuroscience': '9LJB6XJV',
 'EEG is better left alone': 'SUMVYN4I',
 'Everything you wanted to ask about EEG but were afraid to get the right answer': 'JRGNMUEC',
 'Extremely simple nonlinear noise-reduction method': 'AQNFL6KM',
 'Is the EEG really "chaotic" in hypsarrhythmia?': '2MT58YPB',
 'Meanfield modeling of propofol-induced changes in spontaneous EEG rhythms': 'J2W4I6CD',
 'NeuroKit2: A Python toolbox for neurophysiological signal processing': 'WCGJVE3E',
 'Parameterizing neural power spectra into periodic and aperiodic components': 'H6MFL7PU',
 'Permutation Entropy: Too Complex a Measure for EEG Time Series?': 'C3HBCGUI',
 'Removing electroencephalographic artifacts by blind source separation': 'AWMYJE77',
 'SCCN: Independent Component Labeling': '5UMTLU6X'}


In [6]:
list(item_dict.keys())

['Permutation Entropy: Too Complex a Measure for EEG Time Series?',
 'EEG is better left alone',
 'Parameterizing neural power spectra into periodic and aperiodic components',
 'DISCOVER-EEG: an open, fully automated EEG pipeline for biomarker discovery in clinical neuroscience',
 'Meanfield modeling of propofol-induced changes in spontaneous EEG rhythms',
 'Removing electroencephalographic artifacts by blind source separation',
 'Everything you wanted to ask about EEG but were afraid to get the right answer',
 'NeuroKit2: A Python toolbox for neurophysiological signal processing',
 'A Tutorial on Independent Component Analysis',
 'A Tutorial on Principal Component Analysis',
 'Extremely simple nonlinear noise-reduction method',
 'Is the EEG really "chaotic" in hypsarrhythmia?',
 'SCCN: Independent Component Labeling']

In [7]:
ITEM_ID = item_dict.get(list(item_dict.keys())[0])
item = zot.item(ITEM_ID)
pprint(item)

{'data': {'DOI': '10.3390/e19120692',
          'ISSN': '1099-4300',
          'abstractNote': 'Permutation entropy (PeEn) is a complexity measure '
                          'that originated from dynamical systems theory. '
                          'Specifically engineered to be robustly applicable '
                          'to real-world data, the quantity has since been '
                          'utilised for a multitude of time series analysis '
                          'tasks. In electroencephalogram (EEG) analysis, '
                          'value changes of PeEn correlate with clinical '
                          'observations, among them the onset of epileptic '
                          'seizures or the loss of consciousness induced by '
                          'anaesthetic agents. Regarding this field of '
                          'application, the present work suggests a relation '
                          'between PeEn-based complexity estimation and '
         

In [8]:
def parse_creators(creators):
    authors = ''
    for author in creators:
        authors += f'{author.get("lastName")}, {author.get("firstName")}; '
    return authors.strip()


def parse_tags(tags):
    tag_string = ''
    for tag in tags:
        tag_string += f'{tag.get("name")}; '
    return tag_string.strip()


def parse_extra_to_citation_key(extra):
    citation_key = ''
    for line in extra.split('\n'):
        if line.startswith('Citation Key: '):
            citation_key = line.split('Citation Key: ')[1].strip()
            return citation_key
    return citation_key


def pdf_path_from_local_pdf_id(local_id: str) -> str:
    storage_path = os.path.join(os.getenv('LOCAL_ZOTERO_PATH'), 'storage', local_id)
    if not (files := os.listdir(storage_path)):
        return ''
    for file in files:
        if file.endswith('.pdf'):
            pdf_path = os.path.join(storage_path, file)
            return pdf_path
    return ''


def parse_links(links):
    attachment = links.get('attachment', {})
    if not attachment.get('attachmentType') == 'application/pdf':
        return ''
    href = attachment.get('href', '')
    local_id = href.split(os.sep)[-1]
    return pdf_path_from_local_pdf_id(local_id)


def parse_item(item: dict) -> dict:
    mdata = item.get('data')
    parsed_dict = {
        'title': mdata.get('title'),
        'authors': parse_creators(mdata.get('creators', [])),
        'abstract': mdata.get('abstractNote'),
        'TAGS': parse_tags(mdata.get('TAGS', [])),
        'citation_key': parse_extra_to_citation_key(mdata.get('extra')),
        'doi': mdata.get('DOI'),
        'date': mdata.get('date'),
        'attachments': parse_links(item.get('links', {})),
    }
    return parsed_dict


pprint(parse_item(item))

{'TAGS': '',
 'abstract': 'Permutation entropy (PeEn) is a complexity measure that '
             'originated from dynamical systems theory. Specifically '
             'engineered to be robustly applicable to real-world data, the '
             'quantity has since been utilised for a multitude of time series '
             'analysis tasks. In electroencephalogram (EEG) analysis, value '
             'changes of PeEn correlate with clinical observations, among them '
             'the onset of epileptic seizures or the loss of consciousness '
             'induced by anaesthetic agents. Regarding this field of '
             'application, the present work suggests a relation between '
             'PeEn-based complexity estimation and spectral methods of EEG '
             'analysis: for ordinal patterns of three consecutive samples, the '
             'PeEn of an epoch of EEG appears to approximate the centroid of '
             'its weighted power spectrum. To substantiate this propo

In [9]:
def local_pdf_id_from_item_id(item_id: str) -> str:
    item = zot.item(item_id)
    links = item.get('links', {})
    attachment = links.get('attachment', {})
    if not attachment.get('attachmentType') == 'application/pdf':
        return ''
    href = attachment.get('href', '')
    local_id = href.split(os.sep)[-1]
    return local_id


def local_pdf_path_from_item_id(item_id: str) -> (str, str):
    """Find the local pdf based on the item_id"""
    local_id = local_pdf_id_from_item_id(item_id)
    return pdf_path_from_local_pdf_id(local_id)


In [10]:
local_pdf_path_from_item_id(ITEM_ID)

'/home/soenke/Zotero/storage/NIJZGAEK/Berger et al. - 2017 - Permutation Entropy Too Complex a Measure for EEG.pdf'

In [12]:
from src.zotero_client import ZoteroClient

client = ZoteroClient()

TypeError: Too many arguments for typing.List; actual 2, expected 1